<a href="https://colab.research.google.com/github/taotaodafeimi/travel_assistant/blob/main/Final_Project_Traveling_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Set-Up

In [ ]:
#install and import mandatory packages

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

!pip install googlemaps
!pip install folium
import googlemaps
from pyomo.environ import *
import folium

In [ ]:
hotel = input('enter your hotel address:')

enter your hotel address:1


In [ ]:
location_list = []
location = input('enter the location you will visit (enter 0 to stop):')
while location != '0':
  location_list.append(location)
  location = input('enter the location you will visit (enter 0 to stop):')


enter the location you will visit (enter 0 to stop):DC
enter the location you will visit (enter 0 to stop):0


#Inputs

In [ ]:
gmaps = googlemaps.Client(key='goole kpi') #I combine Google map API to get the location coordinates and find the shortest path

hotel_address = '775 12th St NW, Washington D.C.' #enter your hotel address. Remember this address will be your start point in the model. For every location, you can enter the name of building or the address of the location.
locations = [hotel_address, 'White House, Washington D.C.', 'Lincoln Memorial, Washington D.C.', 'Washington Monument, Washington D.C.','United States Capitol, Washington D.C.'] #enter the your plan

# get coordinates for each location using Google Maps API
coordinates_dict = {} #create a dict to store the coordinates for each loc.
for loc in locations:
    geocode_result = gmaps.geocode(loc) #get coordinates for each locoation
    if geocode_result: #make sure we only append the correct location.
        location = geocode_result[0]['geometry']['location'] #get coordinates for each locoation
        coordinates_dict[loc] = (location['lat'], location['lng']) #enter latitude and longitude for each location into coordinates_dict variable

# use Google Maps API to create the distance matrix using driving mode.
distance_matrix = gmaps.distance_matrix(locations, locations, mode="driving")

# Extract distance matrix
D = [[row['elements'][col]['distance']['value'] for col in range(len(locations))]
        for row in distance_matrix['rows']]

n = len(locations)
D

[[0, 1196, 3065, 1772, 3711],
 [918, 0, 2692, 1575, 4681],
 [2890, 2183, 0, 1330, 5058],
 [1772, 1446, 1330, 0, 3126],
 [4823, 5825, 7717, 4015, 0]]

#TSP Model

In [ ]:
# create TSP Model
model = ConcreteModel()
model.x = Var(range(n), range(n), domain=Binary) # model.x[i][j] represents whether the route will go directly from location i to location j. If model.x[i][j] is 1, it means the route includes going from i to j directly; if it's 0, the route does not include this path.
model.u = Var(range(n), domain=NonNegativeReals) # make sure each location is visited one time

# Objective: Minimize the total distance
model.obj = Objective(expr=sum(D[i][j] * model.x[i, j] for i in range(n) for j in range(n) if i != j), sense = minimize)

# Constraints
model.constraints = ConstraintList()

# Each location is visited exactly once
for j in range(n):
    if j != 0:  # Ignore the hotel (starting point) for this constraint
        model.constraints.add(sum(model.x[i, j] for i in range(n) if i != j) == 1)
for i in range(n):
    if i != 0:  # Ignore the hotel (starting point) for this constraint
        model.constraints.add(sum(model.x[i, j] for j in range(n) if i != j) == 1)

# Sub-tour elimination
for i in range(1, n):
    for j in range(1, n):
        if i != j:
            model.constraints.add(model.u[i] - model.u[j] + n * model.x[i, j] <= n - 1)

# Solve the model
solver = SolverFactory('cbc')
result = solver.solve(model)

# Check if the solution is optimal
if result.solver.status == SolverStatus.ok and result.solver.termination_condition == TerminationCondition.optimal:
    # Extract the sequence of locations from the model's solution
    route_sequence = []
    for i in range(n):
        for j in range(n):
            if i != j and value(model.x[i, j]) > 0.5:
                route_sequence.append((i, j))

# Order the locations and calculate the total distance
    ordered_locations = []
    next_location = 0
    total_distance = 0
    while len(ordered_locations) < n:
        for (i, j) in route_sequence:
            if i == next_location:
                ordered_locations.append(locations[i])
                total_distance += D[i][j]
                next_location = j
                break

    # Reorder coordinates based on the route
    ordered_coordinates = [coordinates_dict[loc] for loc in ordered_locations]

    # Create a map and set the first location is the coordinate of the hotel
    map = folium.Map(location=ordered_coordinates[0], zoom_start=15)

    # Add a green marker for the start point
    folium.Marker(ordered_coordinates[0],tooltip="Click me!",popup=ordered_locations[0],icon=folium.Icon(color='green', icon='play')).add_to(map)

    # Add markers for the intermediate points
    for i, point in enumerate(ordered_coordinates[1:-1], start=1):
        folium.Marker(point,tooltip="Click me!",popup=ordered_locations[i]).add_to(map)

    # Add a red marker for the end point
    folium.Marker(ordered_coordinates[-1],tooltip="Click me!",popup=ordered_locations[-1],icon=folium.Icon(color='red', icon='flag')).add_to(map)

    # Add lines for the route
    for i in range(len(ordered_coordinates) - 1):
        folium.PolyLine([ordered_coordinates[i], ordered_coordinates[i + 1]], color='blue').add_to(map)
    #convert meter to miles
    meter_to_miles = 1 / 1609.34
    print(f"Ordered Locations: {ordered_locations}")
    total_distance = total_distance*meter_to_miles
    print(f"Total Distance: {total_distance} meters")
else:
    print("No optimal solution found.")


Ordered Locations: ['775 12th St NW, Washington D.C.', 'United States Capitol, Washington D.C.', 'Washington Monument, Washington D.C.', 'Lincoln Memorial, Washington D.C.', 'White House, Washington D.C.']
Total Distance: 7.554028359451702 meters


#Shortest Path Map

In [ ]:
#show the map
map